# Práctica 4: Programación en Apache Spark

Se propone la realización de 4 scripts. Los scripts deben realizarse usando Notebooks o bien como scripts que se puedan enviar con `spark-submit`.

### Normas:
- Los scripts deben incluir comentarios que expliquen los pasos realizados.
- La salida de los scripts debe seguir el formato indicado en cada uno de los ejercicios (incluyendo el nombre y orden de las columnas).
- Se debe entregar un fichero comprimido con los scripts de la práctica debidamente comentados.
- **TAMBIÉN** un pequeño documento que muestre capturas de la ejecución de los scripts **dentro del clúster**.

## Ejercicio 1

Extraer información de los ficheros de StackOverflow (`Posts.parquet`, `Users.parquet`, `Comments.parquet`). Crear un script que haga lo siguiente:

 a. A partir del fichero `Posts.parquet` obtener el número de respuestas (y la suma de sus `Score`) que ha recibido cada pregunta (posts con `PostTypeId` = 1). Para ello, cuenta cuántos posts con `PostTypeId` = 2 tienen como `ParentId` el `Id` de cada pregunta. Debes obtener un DataFrame de la siguiente forma en el fichero `dfRespuestas.parquet`:

|QuestionId|NRespuestas|ScoreTotal|
|---------:|----------:|---------:|
|4        |13         |120      |
|6        |26         |300      |
|9        |33         |400      |
|11       |15         |150      |
|13       |8          |80       |
|14       |3          |30       |
|17       |6          |60       |


b. A partir del fichero `Posts.parquet`, crear un DataFrame que contenga el Id de la pregunta, el `OwnerUserId` y el año de creación (extraído de `CreationDate`), descartando el resto de campos. Ese DataFrame debe tener la siguiente forma, y estar en el fichero `dfPreguntas.parquet`:

|QuestionId|UserId|Año |
|---------:|-----:|---:|
|4         |8     |2008|
|6         |9     |2008|
|9         |1     |2008|
|11        |1     |2008|
|13        |9     |2008|
|14        |11    |2008|
|17        |2     |2008|
|22        |9     |2008|

### Requisitos

- Ambos DataFrames se deben salvar en formato Parquet con compresión gzip. Comprueba el número de particiones de cada DataFrame y el número de ficheros generados.
- El script debe aceptar argumentos en línea de comandos, es decir, para su ejecución se debe poder indicar la ruta al fichero de entrada y el nombre de los directorios de salida. Por ejemplo, para la ejecución en local:

```bash
spark-submit --master 'local[*]' --num-executors 4 --driver-memory 4g e1.py Posts.parquet dfRespuestas.parquet dfPreguntas.parquet
```

### Ejemplo:

```python
#! /usr/bin/env python3
from pyspark.sql import SparkSession
import sys
#
# Script para extraer información del fichero Posts.parquet de StackOverflow. 
# a) Obtener el número de respuestas que ha recibido cada pregunta.
#    Debes obtener un DataFrame de la siguiente forma:
#   +----------+-----------+----------+
#   |QuestionId|NRespuestas|ScoreTotal|
#   +----------+-----------+----------+
#   |    4     |     13    |    120   |
#   |    6     |     26    |    300   |
#   |    9     |     33    |    400   |
#   |   11     |     15    |    150   |
#   |   13     |      8    |     80   |
#
# b) A partir del fichero Posts.parquet, crear un DataFrame que contenga el Id de la pregunta, 
# el OwnerUserId y el año de creación, descartando el resto de campos del fichero.
# Ese DataFrame debe tener la siguiente forma:
#
#   +----------+------+----+ 
#   |QuestionId|UserId|Año |
#   +----------+------+----+
#   |    4     |   8  |2008|
#   |    6     |   9  |2008|
#   |    9     |   1  |2008|
#   |   11     |   1  |2008|
#   |   13     |   9  |2008|
#
# Ejecutar en local con:
# spark-submit --master 'local[*]' --driver-memory 4g e1.py Posts.parquet dfRespuestas.parquet dfPreguntas.parquet
# Ejecución en un cluster YARN:
# spark-submit --master yarn --num-executors 8 --driver-memory 4g e1.py Posts.parquet dfRespuestas.parquet dfPreguntas.parquet

def main():
    # Comprueba el número de argumentos
    # sys.argv[1] es el primer argumento, sys.argv[2] el segundo, etc.
    if len(sys.argv) != 4:
        print(f"Uso: {sys.argv[0]} Posts.parquet dfRespuestas.parquet dfPreguntas.parquet")
        exit(-1)

    spark: SparkSession = SparkSession\
        .builder\
        .appName("Ejercicio 1 de Diego")\
        .getOrCreate()

    # Cambio la verbosidad para reducir el número de
    # mensajes por pantalla
    spark.sparkContext.setLogLevel("FATAL")
    # Código del programa
    ...

if __name__ == "__main__":
    main()
```

**Nota:** Para hacer pruebas más rápidamente podéis hacer un sampleo del fichero `Posts.parquet` y trabajar con una versión más reducida.

### Ayuda en la realización del ejercicio:

- Comprueba que tienes acceso a los ficheros
- Cargamos datos en dataframes

En el shell de Unix:

```bash
# Descarga del fichero Posts.parquet
wget -qq https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/Posts.parquet

# También puedes descargar Users, Comments y Tags si los necesitas
wget -qq https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/Users.parquet
wget -qq https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/Comments.parquet
wget -qq https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/Tags.parquet

# Listamos ficheros
ls -lh *.parquet
```

Para cargar los datos del fichero `Posts.parquet` podéis usar el siguiente código:

```python
from pyspark.sql import DataFrame

def load_posts_data(spark: SparkSession, path_posts: str) -> DataFrame:
    posts: DataFrame = (spark
        .read
        .format("parquet")
        .load(path_posts))
    
    posts.printSchema()
    posts.show(10)
    print(f"Total de posts: {posts.count()}")
    
    return posts
```

## Ejercicio 2

Script que, a partir de los datos en Parquet de la práctica anterior, obtenga para cada usuario y para cada año el total de preguntas realizadas, el total de respuestas recibidas por todas sus preguntas, el total de comentarios a sus preguntas, la media de respuestas por pregunta y el máximo número de respuestas recibidas en una pregunta.
- Obtener solo aquellos casos en los que existan valores en ambos DataFrames (inner join).
- Cada usuario debe aparecer con su `DisplayName`, obtenido del fichero `Users.parquet`.
- Los comentarios se obtienen del fichero `Comments.parquet`, que tiene una columna `PostId` con el Id del post (pregunta o respuesta) que se comenta.
- El DataFrame generado debe estar ordenado por el máximo número de respuestas, usuario y año.

Ejemplo de salida:


|Usuario      |Año |NumPreguntas|TotalRespuestas|TotalComentarios|MediaRespuestas   |MaxRespuestas|
|:------------|---:|-----------:|--------------:|---------------:|-----------------:|------------:|
|Jon Skeet    |2008|156         |1247           |234             |7.99              |45           |
|Marc Gravell |2008|89          |456            |123             |5.12              |32           |
|Mehrdad      |2009|34          |234            |45              |6.88              |28           |
|Joel         |2008|45          |189            |67              |4.20              |23           |
|Greg Hewgill |2008|67          |345            |89              |5.15              |19           |
|... |... |...  |...    |...   |...   |...|


### Requisitos
- El DataFrame obtenido se debe guardar en un único fichero CSV sin comprimir y con cabecera.
- Como en el caso anterior, el script debe aceptar argumentos en línea de comandos.

**Ayuda en la realización del ejercicio:**

- Lectura de fichero parquet.
- Cargamos el fichero con los códigos del país en un diccionario.
- Guardar un DataFrame en un único fichero CSV sin comprimir y con cabecera.

```python
from pyspark.sql import DataFrame

# Leo el fichero de respuestas
dfRespuestas: DataFrame = (spark.read.format("parquet")
               .option("mode", "FAILFAST")
               .load("dfRespuestas.parquet"))

# Leo el fichero de preguntas
dfPreguntas: DataFrame = (spark.read.format("parquet")
               .option("mode", "FAILFAST")
               .load("dfPreguntas.parquet"))

# Leo el fichero de usuarios
dfUsers: DataFrame = (spark.read.format("parquet")
               .option("mode", "FAILFAST")
               .load("Users.parquet"))

# Leo el fichero de comentarios
dfComments: DataFrame = (spark.read.format("parquet")
               .option("mode", "FAILFAST")
               .load("Comments.parquet"))

dfUsers.printSchema()
dfUsers.show(10)

# Lo guardamos como un único fichero CSV
(dfRespuestas.coalesce(1)
       .write.format("csv")
       .mode("overwrite")
       .option("header", True)
       .save("resultado_e2"))
```

## Ejercicio 3

Obtener a partir de los ficheros Parquet creados en el ejercicio 1 y del fichero `Tags.parquet` un DataFrame que proporcione, para un grupo de tags especificados, las preguntas ordenadas por número de respuestas, de mayor a menor, junto con una columna que indique el rango (posición de la pregunta en ese tag/año según las respuestas obtenidas):

La salida del script debe ser como sigue:

|Tag    |Año |QuestionId|NRespuestas|Rango|
|:------|---:|---------:|----------:|----:|
|python |2008|155       |89         |1    |
|python |2008|231       |67         |2    |
|python |2008|456       |45         |3    |
|python |2008|678       |43         |4    |
|python |2008|890       |38         |5    |
|python |2009|1234      |92         |1    |
|python |2009|2345      |78         |2    |
|python |2009|3456      |65         |3    |
|... |... |...     |...   |...|
|java   |2008|234       |76         |1    |
|java   |2008|345       |68         |2    |
|java   |2008|567       |54         |3    |
|... |... |...     |...   |...|


### Requisitos
* El DataFrame debe de estar ordenado por tag y año (ascendente) y número de respuestas (descendente).
* Utilizad funciones de ventana para obtener el rango.
* La salida debe guardarse en un único fichero CSV sin comprimir y con cabecera.
* Como en los casos anteriores, el script debe aceptar argumentos en línea de comandos, es decir, para su ejecución deberíamos poder indicar la ruta a los directorios de entrada creados en la práctica 1, la lista de tags a analizar (separados por coma) y el nombre del directorio de salida.

**Nota:** El fichero `Tags.parquet` tiene las columnas `Id` (del post) y `TagName`. Una pregunta puede tener múltiples tags asociados.

## Ejercicio 4

Obtener a partir del fichero Parquet con la información de preguntas (QuestionId, UserId y Año) creado en el ejercicio 1, un DataFrame que nos muestre el número de preguntas realizadas por cada usuario (dado por su nombre, `DisplayName` de `Users.parquet`) por cada tag y por cada año. Adicionalmente, debe mostrar el aumento o disminución del número de preguntas para cada usuario con respecto al año anterior.

El DataFrame generado tiene que ser como este:

|Usuario      |Tag    |Año |NPreguntas|Dif |
|:------------|:------|---:|---------:|---:|
|Jon Skeet    |python |2008|12        |0   |
|Jon Skeet    |python |2009|18        |6   |
|Jon Skeet    |python |2010|15        |-3  |
|Jon Skeet    |java   |2008|33        |0   |
|Jon Skeet    |java   |2009|49        |16  |
|Jon Skeet    |java   |2010|19        |-30 |
|Marc Gravell |python |2008|23        |0   |
|Marc Gravell |python |2009|28        |5   |
|Marc Gravell |python |2010|31        |3   |
|Joel Coehoorn|c#     |2008|12        |0   |
|Joel Coehoorn|c#     |2009|45        |33  |
|Greg Hewgill |java   |2009|8         |0   |
|Greg Hewgill |java   |2010|15        |7   |
|... |...   |...   |...     |...|

### Requisitos
* El DataFrame debe de estar ordenado por Usuario, Tag y año.
* Para obtener la diferencia con el año anterior, utilizad funciones de ventana (window functions).
* La salida debe guardarse en un único fichero CSV sin comprimir y con cabecera.
* Como en los casos anteriores, el script debe aceptar argumentos en línea de comandos, es decir, para su ejecución deberíamos poder indicar la ruta al directorio de entrada creado en la práctica 1 y el nombre del directorio de salida.
* Opcionalmente, se puede añadir un filtro para mostrar solo los usuarios que tengan al menos un número mínimo de preguntas totales (por ejemplo, 10 o más preguntas en total).